<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : Pyxel - Snake</h1>

[Pyxel](https://github.com/kitao/pyxel/blob/main/docs/README.fr.md) est un moteur de jeu rétro pour Python

Dans cette activité, nous allons créer un petit jeu classique : [Snake](https://fr.wikipedia.org/wiki/Snake_(genre_de_jeu_vid%C3%A9o)).

## Cahier des charges
* le serpent se meut automatiquement, on peut le déplacer avec les flèches du clavier.
* s'il mange la pomme, il grandit et celle-ci réapparait dans une case vide
* s'il quitte l'écran ou se mord, il meurt, et le jeu s’arrête

## Principes généraux des jeux vidéos
Un jeu vidéo peut être résumé ainsi :
Une boucle infinie fait progresser le jeu :
A chaque tour :
    1. On écoute les interactions du joueur
    2. On met à jour l'état du jeu
    3. On dessine les éléments à l'écran,
    4. On attend quelques millisecondes
    
Dans Pyxel, la boucle infinie est implicite, et l’attente des quelques millisecondes déjà prise en charge (pas besoin de s’en occuper).
Des fonctions prédéfinies gèrent les actions 2 (`update()`) et 3 (`draw()`)

| action                          | fonction Pyxel |
|---------------------------------|----------------|
| Mettre à jour l’état du jeu     | `update()`     |
| Dessiner les éléments à l’écran | `draw()`       |

Au début du programme, on crée la fenêtre du jeu : 

In [ ]:
pyxel.init(400, 400, title="snake")

A la fin du programme, on lance l’exécution du jeu :

In [ ]:
pyxel.run(update, draw)

qui fait appel aux deux fonctions prédéfinies, qui seront appelées 20 fois par seconde.
Il existe de nombreuses méthodes toutes faites permettant de dessiner, écrire du texte.  
Les couleurs sont désignées par des entiers de 0 à 15 (0 désignant le noir).

| action                                | fonction Pyxel                 |
|---------------------------------------|--------------------------------|
| Effacer l’écran et le remplir de noir | `cls(0)`                      |
| Détection d’interactions utilisateurs | `btn(pyxel.KEY_RIGHT)`         |
| Ecrire du texte                       | `text(50, 64, 'GAME OVER', 7)` |
| Dessiner un rectangle                 | `rect(x, y, long, larg, 1)`    |


En Pyxel, on utilise généralement des variables globales qui sont définies à la racine du script et sont mises à jour dans `update()`.  
*Ce n'est pas une bonne pratique en programmation mais c'est facile et parfois nécessaire pour la programmation événementielle*

Pour préciser que la fonction a le droit de modifier une variable globale, par exemple le score : on écrira `global score`.

### Version 1 : dessiner le serpent
#### La grille
Les cases seront représentées par des coordonnées. 
L’origine est en haut à gauche. 
On commence à zéro, la 1ère coordonnée est l’abscisse (numéro de colonne) et la seconde l’ordonnée (numéro de ligne).  


Exemple : ici, la grille a pour dimensions 200x160 pixels, et 10 cases par 8.

On définit alors les variables `HEIGHT`, `WIDTH`, `CASE` (en majuscules car ce sont des constantes).

Puis on peut créer la fenêtre avec `pyxel.init()`.

In [ ]:
import pyxel

# Constantes du jeu
TITLE = "Snake"
WIDTH = 200
HEIGHT = 160
CASE = 20

pyxel.init(WIDTH, HEIGHT, title=TITLE)

def draw():
    pyxel.cls(0) # Effacer l'écran puis le remplir de noir

#### Le serpent
Le serpent est représenté par une variable double liste : `snake = [[3, 3], [2, 3], [1, 3]]`, définie au début du programme (après `pyxel.init()`)
Le premier élément est sa tête, elle est en `[3, 3]` ensuite vient son corps.

#### Dessiner le serpent
Pour dessiner sur l’écran les cases du serpent, on utilise la méthode `pyxel.rect(x, y, L, l, color)`.
* `x` et `y` sont les coordonnées du coin supérieur gauche, `L` et `l` les dimensions du rectangle.
* `color` est un indice entre 0 et 15 désignant une couleur de la palette prédéfinie Pyxel. 
Les instructions suivantes seront placées dans la fonction `draw()`.

In [ ]:
# Dessiner le corsp en vert
for anneau in snake[1:]:
        x, y = anneau[0], anneau[1]
        pyxel.rect(x * CASE, y * CASE, CASE, CASE, 11) # 11 est le vert
    x_head, y_head = snake[0]
    pyxel.rect(x_head * CASE, y_head * CASE, CASE, CASE, 9) # Dessiner la tête en orange

#### Ecrire le score
Au début, la variable globale `score` vaut `0` (à définir au même endroit que la variable `snake`, au niveau principal du programme, à l’extérieur de toute fonction).  
On la mettra à jour plus tard dans la fonction `update()`, mais on peut déjà écrire le score initial sur la fenêtre, par une instruction dans la fonction `draw()`.

In [ ]:
pyxel.text(4, 4, f"SCORE : {score}", 7) # 7 est la couleur blanche

Enfin, on écrit une fonction `update()` pour l’instant vide, et on lance le jeu avec `pyxel.run(update, draw)`.

In [ ]:
def update():
    pass

pyxel.run(update, draw)

Le serpent est dessiné !

### Version 2 : animer le serpent
Jusqu’ici, le serpent ne bougeait pas. On va l’animer un peu. 
#### Déplacer le serpent « tout droit »
Pour commencer, on va supposer que la direction de déplacement du serpent est `direction = [1 ,0]`.  
On ajoute la variable globale `direction` au début.

In [ ]:
# La nouvelle tête est l'ancienne, d'placée dans la direction
head = [snake[0][0] + direction[0], snake[0][1] + direction[1]] 

# On l'insère au début
snake.insert(0, head)

On efface le dernier élément de `snake` pour terminer le mouvement : `snake.pop()`.

A faire : Intégrer ces instructions dans la fonction `update()` qui est appelée automatiquement par Pyxel 30 fois par seconde, et lancer le programme. 

#### Ralentir le jeu
30 images par secondes (ou Frames Per Second FPS), ça donne une bonne fluidité d’affichage, mais ça fait quand même trop rapide pour le mouvement du serpent. Pour ralentir, on va utiliser le compteur de frames intégré à Pyxel, en effectuant le mouvement par exemple uniquement tous les 15 frames.

On rajoute la constante `FRAME_REFRESH = 15` au début avec les constantes, puis dans la fonction `update()` on met le mouvement au sein d’un test.  
Vérifiez : le mouvement est beaucoup plus lent !

In [ ]:
def update():
    if pyxel.frame_count % FRAME_REFRESH == 0:
        head = [snake[0][0] + direction[0], snake[0][1] + direction[1]]
        snake.insert(0, head)
        snake.pop()    

#### Changer la direction du serpent
Cela va se faire dans la fonction `update()` en *écoutant* les interactions du joueur (quand il tape sur une touche du clavier) avec `pyxel.btn`.  
NB : pour avoir le droit de modifier la variable `direction` au sein de la fonction `update()`, elle doit être bien déclarée comme globale dans cette fonction.

In [ ]:
if pyxel.btn(pyxel.KEY_ESCAPE):
    pyxel.quit()
elif pyxel.btn(pyxel.KEY_RIGHT):
    direction = [1, 0]
elif pyxel.btn(pyxel.KEY_LEFT):
    direction = [-1, 0]

Le serpent tourne !

### Faire mourir le serpent
Dans notre version du jeu : le serpent meurt lorsqu'il se mord la queue, ou lorsqu'il quitte l'écran. Dans ce cas, le jeu s’arrête, et on quitte la fenêtre.  

Pour savoir si la tête du serpent a touché son corps : on teste si les coordonnées de la tête correspondent à un anneau déjà existant du serpent. 

Pour savoir si la tête du serpent « sort » de la fenêtre : on doit vérifier plusieurs conditions :

In [ ]:
if head in snake[1:] or head[0] < 0 or head[0] > WIDTH / CASE - 1 or head[1] < 0 or head[1] > HEIGHT / CASE - 1:
    pyxel.quit()

D’où la condition multiple dans la fonction `update()`.  

Vérifiez son fonctionnement : pour le cas où il se mord la queue, vous aurez besoin de définir au début un serpent plus long.

Le serpent peut mourir !

### Version 4 : manger la pomme… et réagir !
On place une pomme (matérialisée par une case rose) au hasard dans la fenêtre. Lorsque le serpent mange la pomme, il grandit d’un anneau (sa queue n’est pas effacée), et le score augmente de 1. 
Variable représentant la pomme : `food = [8, 3]` (au début, on la place arbitrairement).

In [ ]:
# La nourriture :
# food est une variable globale
x_food, y_food = food
pyxel.rect(x_food * CASE, y_food * CASE, CASE, CASE, 8) # 8 est la couleur rose 

Pour replacer une nouvelle pomme, on tire au hasard des coordonnées dans la grille. Pour cela, on a besoin de la fonction `randint`.  
`randint` doit être importée depuis le module `random`, au tout début du programme.  
`randint` renvoie un entier aléatoire compris entre `a` et `b`. 
On recommence jusqu’à ce que ces coordonnées soient OK (pas *dans le corps du serpent*)

In [ ]:
while food in snake: #necessaire de relancer plusieurs fois si on n'a pas de chance
    food = [randint(0, WIDTH/CASE - 1), randint(0, HEIGHT/CASE - 1)]
# sortie du while : on a trouvé une nouvelle case pour la pomme

Le serpent grandit et la pomme réapparaît !

### Extensions possibles
A ce stade, le jeu est terminé ! Plusieurs améliorations sont possibles : 
* au lieu de quitter si le serpent meurt
* relancer instantanément une nouvelle partie
* conserver un high score (tant qu'on ne quitte pas le jeu puis de manière persistante en l'écrivant dans un fichier)
* améliorer les graphismes
* ajouter du son
* ...